# https://www.kaggle.com/datasets/oyekanmiolamilekan/fake-employee-dataset

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import hashlib
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
df = pd.read_csv("fake_employee.csv")
df.head()

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,education_level,certifications,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,Master's Degree,team,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,PhD,"whether, some, measure","perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,PhD,"job, week","tend, official, red",4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,PhD,"position, similar","question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,Bachelor's Degree,"table, seem","how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331


In [165]:
df.info()
df.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   employee_id          1000 non-null   int64 
 1   name                 1000 non-null   object
 2   gender               1000 non-null   object
 3   dates_of_birth       1000 non-null   object
 4   email                1000 non-null   object
 5   phone_number         1000 non-null   object
 6   address              1000 non-null   object
 7   department           1000 non-null   object
 8   job_titles           1000 non-null   object
 9   manager_id           1000 non-null   int64 
 10  hire_date            1000 non-null   object
 11  salary               1000 non-null   int64 
 12  employment_status    1000 non-null   object
 13  employee_type        1000 non-null   object
 14  education_level      1000 non-null   object
 15  certifications       1000 non-null   object
 16  skills 

,employee_id,manager_id,salary,performance_ratings
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,5527.310000,5.454000,81531.299000,2.991000
std,2546.946895,2.837241,23216.141548,1.406021
min,1022.000000,1.000000,40027.000000,1.000000
25%,3351.500000,3.000000,61903.750000,2.000000
50%,5559.000000,5.000000,82902.000000,3.000000
75%,7732.750000,8.000000,101614.750000,4.000000
max,9995.000000,10.000000,119987.000000,5.000000


# Functions

In [166]:

def preprocess_for_ml(df_in):
    df = df_in.copy()
    df = df.drop(columns=['salary'])
    y = df['rounded_salary']
    X = df.drop(columns=['rounded_salary'])  # Drop the target column
    
    # Encode categorical variables
    for col in X.select_dtypes(include='object').columns:
        X[col] = LabelEncoder().fit_transform(X[col])
        
    return X, y


In [167]:

# Function to train and evaluate models
def train_and_evaluate(model, X_train, X_test, y_train, y_test, label=''):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Model ({label}) - MAE: {mae:.2f}")
    return mae


In [168]:

# Training and evaluation function for all models
def evaluate_models(X_train, X_test, y_train, y_test):
    models = [
        (RandomForestRegressor(random_state=42), "Random Forest"),
        (LinearRegression(), "Linear Regression"),
        (DecisionTreeRegressor(random_state=42), "Decision Tree"),
        (GradientBoostingRegressor(random_state=42), "Gradient Boosting")
    ]
    
    results = {}
    for model, label in models:
        mae = train_and_evaluate(model, X_train, X_test, y_train, y_test, label)
        results[label] = mae

    return results


In [169]:
# Use the original dataset (with PII)
df_original = df.copy()



In [170]:
df_original

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,education_level,certifications,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,Master's Degree,team,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,PhD,"whether, some, measure","perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,PhD,"job, week","tend, official, red",4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,PhD,"position, similar","question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,Bachelor's Degree,"table, seem","how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,PhD,something,"week, with, character, source",4,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,PhD,"study, election, game","level, price, measure, nor",2,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,Associate Degree,"nice, reason, plan","relate, like, production",5,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,High School,"receive, product, create","budget, we, high, industry",4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033


In [178]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a countplot for the rounded salary (assuming 'rounded_salary_5000' is the column)
sns.countplot(data=df, x='salary')
plt.title('Distribution of Rounded Salary (5,000)')
plt.xlabel('Rounded Salary')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [171]:

# X, y = preprocess_for_ml(df_original)  # This step should be from your previous code

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Evaluate models
# model_results = evaluate_models(X_train, X_test, y_train, y_test)

# # Print the comparison of MAE results
# print("\nModel Comparison - MAE:")
# for model_name, mae in model_results.items():
#     print(f"{model_name}: {mae:.2f}")


In [172]:
# Round salary to the nearest 5,000
df['rounded_salary'] = (df['salary'] // 10000) * 10000


In [173]:
df['salary']

0       77367
1       81869
2       97907
3      110034
4       70091
        ...  
995     85160
996    109359
997     70527
998     45389
999     64225
Name: salary, Length: 1000, dtype: int64

In [174]:
df['rounded_salary']

0       70000
1       80000
2       90000
3      110000
4       70000
        ...  
995     80000
996    100000
997     70000
998     40000
999     60000
Name: rounded_salary, Length: 1000, dtype: int64

In [176]:
df

,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,certifications,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts,rounded_salary
0,6612,Martha Hutchinson,Male,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",Finance,Administrator,10,...,team,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",Hollandberg,9-5,Active,Alicia Payne: 001-469-835-3679x07176,70000
1,4793,Meghan Cervantes,Non-Binary,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",Finance,Mental health nurse,8,...,"whether, some, measure","perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",East Sabrinafurt,9-5,Active,Brandy Lopez: +1-642-314-0561x6900,80000
2,8832,April Hanna,Male,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",Finance,Field seismologist,6,...,"job, week","tend, official, red",4,Necessary Congress real.,"stop, woman",New Garyfurt,9-5,Active,Craig Rivera: 267.574.5240x08971,90000
3,7044,George Yang,Non-Binary,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",Engineering,"Nurse, children's",5,...,"position, similar","question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",Christophermouth,12-8,On Leave,Lauren Davis: 423.534.7439,110000
4,4498,Joseph Patrick,Male,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",Engineering,Meteorologist,2,...,"table, seem","how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",South Jamestown,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331,70000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,Female,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,IT,Hydrogeologist,5,...,something,"week, with, character, source",4,Interesting similar child. Here piece management.,surface,Nicholasburgh,12-8,Active,Joseph Willis: 001-520-862-0025x48408,80000
996,3027,Jackie Church,Male,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,HR,"Accountant, chartered public finance",10,...,"study, election, game","level, price, measure, nor",2,History few news nature agent need human. Vote...,"floor, history",East Michael,12-8,Active,Dawn Wright: 001-650-318-1070,100000
997,3573,Maria Watson,Non-Binary,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",HR,Pension scheme manager,2,...,"nice, reason, plan","relate, like, production",5,Future benefit quite someone. Half among under...,usually,Heathertown,12-8,Active,Trevor Spencer: 882.361.6741,70000
998,2062,Damon Taylor,Female,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",Marketing,"Editor, magazine features",9,...,"receive, product, create","budget, we, high, industry",4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",West Beverly,12-8,Active,Marilyn Arias: 902-310-6452x4033,40000


In [177]:
X, y = preprocess_for_ml(df)  # This step should be from your previous code

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluate models
model_results = evaluate_models(X_train, X_test, y_train, y_test)

# Print the comparison of MAE results
print("\nModel Comparison - MAE:")
for model_name, mae in model_results.items():
    print(f"{model_name}: {mae:.2f}")

Model (Random Forest) - MAE: 19387.50
Model (Linear Regression) - MAE: 18993.83
Model (Decision Tree) - MAE: 25250.00
Model (Gradient Boosting) - MAE: 19542.91

Model Comparison - MAE:
Random Forest: 19387.50
Linear Regression: 18993.83
Decision Tree: 25250.00
Gradient Boosting: 19542.91


# K-1 Anonymity

In [4]:
import pandas as pd

# Sample DataFrame 'df' with your data loaded

k = 2  # k-anonymity level

QIDs = ['gender', 'city', 'department', 'job_titles']

# Example of simple generalization functions:
def generalize_job_title(title):
    # Group similar job titles into broader categories (example)
    if 'manager' in title.lower():
        return 'manager'
    elif 'engineer' in title.lower():
        return 'engineer'
    elif 'nurse' in title.lower():
        return 'healthcare'
    else:
        return 'other'

def generalize_hire_date(date_str):
    # Convert to year only (reduce precision)
    return pd.to_datetime(date_str).year if pd.notnull(date_str) else 'unknown'

# Apply generalization
df['job_titles_gen'] = df['job_titles'].apply(generalize_job_title)
df['hire_year'] = df['hire_date'].apply(generalize_hire_date)

# Define generalized QIDs for k-anonymity
gen_QIDs = ['gender', 'city', 'department', 'job_titles_gen', 'hire_year']

# Step 1: Group by generalized QIDs and count group sizes
group_sizes = df.groupby(gen_QIDs).size().reset_index(name='counts')

# Step 2: Find groups with counts < k
small_groups = group_sizes[group_sizes['counts'] < k]

# Step 3: Suppress QID values in rows belonging to small groups
def suppress_rows(row):
    # Check if this row's QID combination is in a small group
    cond = ((small_groups[gen_QIDs] == row[gen_QIDs]).all(axis=1)).any()
    if cond:
        # Suppress QID values (replace with '*')
        for col in gen_QIDs:
            row[col] = '*'
    return row

df = df.apply(suppress_rows, axis=1)

# Now df satisfies 2-anonymity on generalized QIDs (approximate)


In [5]:
df


,employee_id,name,gender,dates_of_birth,email,phone_number,address,department,job_titles,manager_id,...,skills,performance_ratings,work_experience,benefits_enrollment,city,work_hours,employee_status,emergency_contacts,job_titles_gen,hire_year
0,6612,Martha Hutchinson,*,1970-11-14,randy09@example.net,708-895-4293x86180,"5301 Norman Flats Apt. 965\r\nNorth Darrell, P...",*,Administrator,10,...,"senior, kitchen, president, call, station",3,Receive determine pass find. Authority laugh r...,"data, cell, people",*,9-5,Active,Alicia Payne: 001-469-835-3679x07176,*,*
1,4793,Meghan Cervantes,*,1992-08-20,matthewsnyder@example.com,001-727-813-2133x30484,"7917 Vincent Trafficway Apt. 254\r\nHallhaven,...",*,Mental health nurse,8,...,"perhaps, knowledge, officer",4,Prevent meet bill computer upon keep receive. ...,"appear, discover",*,9-5,Active,Brandy Lopez: +1-642-314-0561x6900,*,*
2,8832,April Hanna,*,1986-04-17,lwood@example.net,+1-758-402-2437x248,"646 Courtney Hollow\r\nSouth Marc, LA 67617",*,Field seismologist,6,...,"tend, official, red",4,Necessary Congress real.,"stop, woman",*,9-5,Active,Craig Rivera: 267.574.5240x08971,*,*
3,7044,George Yang,*,2001-03-15,ellen93@example.net,001-682-318-3829x45513,"3930 Kelly Wells Apt. 574\r\nPaulton, OR 22142",*,"Nurse, children's",5,...,"question, their, health, single, whole",2,Yard whose different trade. Free foreign quest...,"product, either",*,12-8,On Leave,Lauren Davis: 423.534.7439,*,*
4,4498,Joseph Patrick,*,1998-10-09,patriciacollins@example.net,001-790-812-0148x64174,"0493 Lane Avenue\r\nSmithburgh, IN 65374",*,Meteorologist,2,...,"how, government, finally, east, strong",4,Agreement tax chair strategy bring kid assume ...,"gas, commercial",*,Night Shift,Terminated,Antonio Ferguson: +1-425-489-9331,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9044,Sandra Carpenter,*,2005-06-10,william72@example.org,591.823.6268,85384 Sullivan Manors Apt. 877\r\nNorth Scottb...,*,Hydrogeologist,5,...,"week, with, character, source",4,Interesting similar child. Here piece management.,surface,*,12-8,Active,Joseph Willis: 001-520-862-0025x48408,*,*
996,3027,Jackie Church,*,1971-11-30,morenowyatt@example.net,(640)334-0503,71547 Heather Summit Suite 814\r\nPort Melinda...,*,"Accountant, chartered public finance",10,...,"level, price, measure, nor",2,History few news nature agent need human. Vote...,"floor, history",*,12-8,Active,Dawn Wright: 001-650-318-1070,*,*
997,3573,Maria Watson,*,1979-12-19,chale@example.org,(398)901-9136,"1218 Gonzales Loop\r\nNorth Christineborough, ...",*,Pension scheme manager,2,...,"relate, like, production",5,Future benefit quite someone. Half among under...,usually,*,12-8,Active,Trevor Spencer: 882.361.6741,*,*
998,2062,Damon Taylor,*,2001-08-08,sharris@example.net,451.687.5692x867,"002 Ward Locks\r\nNorth Kimberly, VI 98935",*,"Editor, magazine features",9,...,"budget, we, high, industry",4,Consider prevent knowledge unit beautiful hold...,"media, shake, treat",*,12-8,Active,Marilyn Arias: 902-310-6452x4033,*,*
